# Скачивание центрированных изображений из google maps на локальное хранилище

In [1]:
import requests
import json

import pandas as pd
import numpy as np
import scipy.stats as ss

from tqdm.notebook import tqdm

import sys
sys.path.append('../utils')
from google_tile_centered_downloader import download_centered_image, get_tile_center_coords
from google_tile_downloader import download_google_map_tile

## Настройка сессии (аналогичо [google_api_sandbox.ipynb](google_api_sandbox.ipynb))

In [ ]:
API_KEY = <>

url = f'https://tile.googleapis.com/v1/createSession?key={API_KEY}'
headers = {'Content-Type': 'application/json'}
data = {
    'mapType': 'satellite',
    'language': 'en-US',
    'region': 'US'
}

response = requests.post(url, json=data, headers=headers)
session_token = json.loads(response.text)['session']

In [3]:
zoom = 17

## Считываем координаты и скачиваем positive samples

In [4]:
positives_df_new = pd.read_csv('../coordinates_data/Eastern_desert_archaeological_structures_NEW.csv', delimiter=',')

In [ ]:
# errors = []
# limit = len(positives_df_new)
# for i in tqdm(range(limit)):
#     id_, name, _, lon, lat, _, _, _, _, _, _, _ = positives_df_new.iloc[i]
#     try:
#         img = download_centered_image(lat=lat, lon=lon, z=zoom, session_token=session_token, api_key=API_KEY)
#         img.save(f'../googlemaps_data/positive_samples/pos_{id_}.png')
#     except Exception as e:
#         print(f'{i=} exception: {e}')
#         errors.append(name)
# print(f'{len(errors)=}')

## Генерируем и сохраняем координаты negative samples

In [5]:
lon_coords = np.array(positives_df_new['x coordina'])
lat_coords = np.array(positives_df_new['y coordina'])

n = len(lon_coords)
N = 200000

In [6]:
np.random.seed(42)
weights = np.random.dirichlet(np.ones(n) * 0.2, size=N)

In [7]:
generated_coords = []
centered_coords = []

for ind in tqdm(range(N)):
    lat = np.dot(lat_coords, weights[ind])
    lon = np.dot(lon_coords, weights[ind])
    generated_coords.append((lat, lon))

    lat_tilecenter, lon_tilecenter = get_tile_center_coords(lat, lon, zoom)
    centered_coords.append((lat_tilecenter, lon_tilecenter))

  0%|          | 0/200000 [00:00<?, ?it/s]

In [8]:
with open('../googlemaps_data/negative_coords.csv', 'w') as f:
    f.write('img_id,lat,lon\n')
    for i, (lat, lon) in tqdm(enumerate(centered_coords), total=N):
        img_id = str(i).zfill(6)
        f.write(f'{img_id},{lat},{lon}\n')

  0%|          | 0/200000 [00:00<?, ?it/s]

## Проверяем что все в порядке

In [14]:
# lat1, lon1 = generated_coords[2]
# lat1, lon1

In [15]:
# download_centered_image(lat=lat1, lon=lon1, z=zoom, session_token=session_token, api_key=API_KEY)

In [16]:
# lat2, lon2 = centered_coords[2]
# lat2, lon2

In [17]:
# download_google_map_tile(lat=lat1, lon=lon1, z=zoom, session_token=session_token, api_key=API_KEY)

In [18]:
# download_google_map_tile(lat=lat2, lon=lon2, z=zoom, session_token=session_token, api_key=API_KEY)

## Cкачиваем negative samples

In [21]:
errors = []

In [20]:
for i in tqdm(range(N)):

    img_id = str(i).zfill(6)
    
    try:
        lat, lon = centered_coords[i]
        local_file_name = f'../googlemaps_data/negative_samples/{img_id}.png'
        img = download_google_map_tile(lat=lat, lon=lon, z=zoom, session_token=session_token, api_key=API_KEY)
        img.save(local_file_name)

    except Exception as e:
        print(f'{img_id=}: Exception {e}')
        errors.append(img_id)
        continue

  0%|          | 0/10 [00:00<?, ?it/s]